## Part 1: With Images

In [1]:
# to download the data set 
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.metrics import jaccard_similarity_score

mnist = input_data.read_data_sets("/Users/aman/Workspace/DataMining/MNIST_data/", one_hot=True)

/Users/aman/.envs/digit_rec/lib/python2.7/site-packages/matplotlib/__init__.py:955: UserWarning: Illegal line #1
	"TkAgg
"
	in file "/Users/aman/.matplotlib/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)


Extracting /Users/aman/Workspace/DataMining/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/aman/Workspace/DataMining/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/aman/Workspace/DataMining/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/aman/Workspace/DataMining/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# getting a brief info about the data
dir(mnist)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getslice__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 'count',
 'index',
 'test',
 'train',
 'validation']

In [4]:
mnist.test.images

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [1]:
def normalize_data(data):
    data = data.copy()
    data = np.divide((data -  np.mean(data, axis=0)),
                       np.std(data, axis=0),
                       where=np.std(data, axis=0)!=0)
    return data

def get_acc(labels,matrix,k=5):
    sim_matrix=matrix.copy()
    res =[0 for i in range(labels.shape[0])]
    #  predictive KNN Logic
    for i in range(labels.shape[0]):
        sim_matrix[i][i]=0
        res[i]=1 if np.argmax(labels[i])==np.argmax(np.sum(labels[np.argpartition(sim_matrix[i], -k)[-k:]],axis=0)) else 0
        if (i% 10000)==0:
            print i
    # res is the list of prediction result 1 for correct and 
    # 0 for false predition    
    return sum(res)*100/labels.shape[0]

In [6]:
%%time
# Zero mean, unit variance : substract the mean, divide by the appropriate value to get variance=1. (required)
test_array = normalize_data(mnist.test.images)

CPU times: user 62.1 ms, sys: 27.5 ms, total: 89.6 ms
Wall time: 91.9 ms


In [7]:
%%time
train_array = normalize_data(mnist.train.images)

CPU times: user 413 ms, sys: 282 ms, total: 695 ms
Wall time: 701 ms


In [8]:
%%time
# simlarity comparision 
sim_train = cosine_similarity(sparse.csr_matrix(mnist.train.images),sparse.csr_matrix(mnist.train.images))

CPU times: user 9min 7s, sys: 3min 14s, total: 12min 22s
Wall time: 17min 57s


In [3]:
%%time
sim_train=[]
with open('/Users/aman/Workspace/DataMining/sim_train.npy','r') as f:
    sim_train  = np.load(f)

CPU times: user 3.38 ms, sys: 14.2 s, total: 14.2 s
Wall time: 31.3 s


In [11]:
with open('/Users/aman/Workspace/DataMining/sim_train','wb') as f:
    np.save(f, sim_train)

In [2]:
%%time
sim_train_e = euclidean_distances(sparse.csr_matrix(mnist.train.images),sparse.csr_matrix(mnist.train.images))

CPU times: user 10min 20s, sys: 5min 11s, total: 15min 32s
Wall time: 25min 19s


In [12]:
%%time
sim_test = cosine_similarity(mnist.test.images,mnist.test.images)

CPU times: user 2.21 s, sys: 254 ms, total: 2.47 s
Wall time: 1.46 s


In [4]:
%%time
resp = get_acc(mnist.train.labels,sim_train)
print 'accurray: ', resp

0
10000
20000
30000
40000
50000
accurray:  97
CPU times: user 51.9 s, sys: 54.1 s, total: 1min 46s
Wall time: 4min 29s


In [13]:
%%time
resp = get_acc(mnist.test.labels,sim_test)
print 'accurray: ', resp

accurray:  96
CPU times: user 1.5 s, sys: 534 ms, total: 2.03 s
Wall time: 2.55 s


In [3]:
np.save('/Users/aman/Workspace/DataMining/sim_train_e', sim_train_e)

(55000,)